## Import necessary packages

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"
from kilt import retrieval
from kilt import kilt_utils as utils
import tasks
# from kilt.retrievers import DPR_connector
import utils
from rouge_score import rouge_scorer
import random
import numpy as np
import torch
torch.set_grad_enabled(False)
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from datasets import load_dataset
import json
from tqdm import tqdm
import opensource

seed = 10
np.random.seed(seed)
random.seed(seed)

In [2]:
import pickle
def write_list(a_list, file_name):
    # store list in binary file so 'wb' mode
    with open(file_name, 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')
def read_list(file_name):
    # for reading also binary mode is important
    with open(file_name, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

In [3]:
def read_chatgpt_results(task):
    retrieved_scores = read_list(f'chatgpt_retrieved_scores_{task}_fewshot.p')
    retrieved_true_scores = read_list(f'chatgpt_retrieved_true_scores_{task}_fewshot.p')
    queries = read_list(f'chatgpt_queries_{task}_fewshot.p')
    answers = read_list(f'chatgpt_true_answers_{task}_fewshot.p')
    chatgpt_true_scores = read_list(f'chatgpt_true_scores_{task}_fewshot.p')
    chatgpt_answers = read_list(f'chatgpt_answers_{task}_fewshot.p')
    chatgpt_passages = read_list(f'chatgpt_passages_{task}_fewshot.p')
    chatgpt_semantics = read_list(f'chatgpt_semantics_{task}_fewshot.p')
    chatgpt_occurances = read_list(f'chatgpt_occurances_{task}_fewshot.p')
    chatgpt_semantic_ids = read_list(f'chatgpt_semantic_ids_{task}_fewshot.p')
    chatgpt_probs = read_list(f'chatgpt_probs_{task}_fewshot.p')
    feasibilities = read_list(f'chatgpt_feasibilities_{task}_fewshot.p')
    
    retrieved_scores_unc = read_list(f'chatgpt_retrieved_scores_unc_{task}_fewshot.p')
    retrieved_true_scores_unc = read_list(f'chatgpt_retrieved_true_scores_unc_{task}_fewshot.p')
    queries_unc = read_list(f'chatgpt_queries_unc_{task}_fewshot.p')
    answers_unc = read_list(f'chatgpt_answers_unc_{task}_fewshot.p')
    passages_unc = read_list(f'chatgpt_passages_unc_{task}_fewshot.p')
    chatgpt_true_scores_unc = read_list(f'chatgpt_true_scores_unc_{task}_fewshot.p')
    chatgpt_answers_unc = read_list(f'chatgpt_answers_unc_{task}_fewshot.p')
    chatgpt_occurances_unc = read_list(f'chatgpt_occurances_unc_{task}_fewshot.p')
    chatgpt_semantic_ids_unc = read_list(f'chatgpt_semantic_ids_unc_{task}_fewshot.p')
    chatgpt_probs_unc = read_list(f'chatgpt_probs_unc_{task}_fewshot.p')
    
    return feasibilities, retrieved_scores, retrieved_true_scores, queries, answers, chatgpt_true_scores, chatgpt_answers, chatgpt_passages, chatgpt_semantics, chatgpt_occurances, chatgpt_semantic_ids, chatgpt_probs, retrieved_scores_unc, retrieved_true_scores_unc, queries_unc, answers_unc, passages_unc, chatgpt_true_scores_unc, chatgpt_answers_unc, chatgpt_occurances_unc, chatgpt_semantic_ids_unc, chatgpt_probs_unc

In [4]:
task = 'nq'

In [5]:
dataset_dpr = tasks.RQA_dpr(task=task)
elements = dataset_dpr.elements

In [6]:
all_queries = [element['question'] for element in elements]

In [7]:
semantic = False
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"],
                                        use_stemmer=True)
if semantic:
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    # setup semantic model
    semantic_tokenizer = \
        AutoTokenizer.from_pretrained("microsoft/deberta-large-mnli")
    semantic_model = \
        AutoModelForSequenceClassification.from_pretrained(
            "microsoft/deberta-large-mnli"
        ).cuda()

In [8]:
feasibilities, retrieved_scores, retrieved_true_scores, queries, answers, chatgpt_true_scores, chatgpt_answers, chatgpt_passages, chatgpt_semantics, chatgpt_occurances, chatgpt_semantic_ids, chatgpt_probs, retrieved_scores_unc, retrieved_true_scores_unc, queries_unc, answers_unc, passages_unc, chatgpt_true_scores_unc, chatgpt_answers_unc, chatgpt_occurances_unc, chatgpt_semantic_ids_unc, chatgpt_probs_unc = \
        read_chatgpt_results(task)

In [9]:
answers = []
for query in queries:
    idx = all_queries.index(query)
    answers.append(elements[idx]['answers'])

In [10]:
answers_semantic = []
probs_semantic = []
for idx_tmp, [true_score, scores, returned_answers] in enumerate(zip(retrieved_true_scores, retrieved_scores, chatgpt_answers)):
    idx = list(scores).index(true_score)
    tmp = returned_answers[idx]
    answers_semantic.append(tmp)

In [13]:
chatgpt_true_scores_semantic = []
skip = []
for idx, [query_tmp, true_answer, generated_answer] in tqdm(enumerate(zip(queries, answers, answers_semantic)), total=len(queries)):

    prompt = utils.get_prompt_template(query_tmp, "", task="Natural Questions")
    semantic_set_ids, semantic_probs, item_occurance = \
                utils.clustering(
        sequences=generated_answer, 
        prompt=None,
        semantic_model=None,
        semantic_tokenizer=None,
        scorer=scorer,
        semantic=False
    )
    true_scores, matched_answer, semantics = utils.processing_answers(
        semantic_set_ids, semantic_probs, 
        item_occurance, true_answer, scorer,
        threshold=0.3
    )
    if len(true_scores) == 0:
        print(idx)
        skip.append(idx)
        continue
    chatgpt_true_scores_semantic.append(true_scores)

 30%|██████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                         | 150/503 [00:01<00:02, 173.50it/s]

133


 52%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                                               | 261/503 [00:02<00:02, 87.45it/s]

273
292


 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                       | 336/503 [00:03<00:01, 119.14it/s]

345


 75%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                   | 376/503 [00:04<00:01, 72.19it/s]

370
385


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 503/503 [00:05<00:00, 91.73it/s]


In [14]:
for count, idx in enumerate(skip):
    retrieved_scores.pop(idx - count)
    retrieved_true_scores.pop(idx - count)
    queries.pop(idx - count)
    answers.pop(idx - count)
    chatgpt_true_scores.pop(idx - count)
    chatgpt_answers.pop(idx - count)
    chatgpt_passages.pop(idx - count)
    chatgpt_occurances.pop(idx - count)
    chatgpt_semantic_ids.pop(idx - count)
    chatgpt_probs.pop(idx - count)

In [15]:
len(retrieved_true_scores)

497

In [16]:
occurances_semantic = []
semantic_ids_semantic = []
probs_semantic = []
for idx, [query_tmp, generated_answers] in tqdm(enumerate(zip(queries, chatgpt_answers)), total=len(queries)):
    probs_tmp = []
    answers_tmp = []
    semantic_id_tmp = []
    occurance_tmp = []
    semantic_tmp = []
    for generated_answer in generated_answers:
        prompt = utils.get_prompt_template(query_tmp, "", task="Natural Questions")
        semantic_set_ids, semantic_probs, item_occurance = \
                utils.clustering(
        sequences=generated_answer, 
        prompt=None,
        semantic_model=None,
        semantic_tokenizer=None,
        scorer=scorer,
        semantic=False
        )
        probs_tmp.append(semantic_probs)
        occurance_tmp.append(item_occurance)
        semantic_id_tmp.append(semantic_set_ids)
    occurances_semantic.append(occurance_tmp)
    semantic_ids_semantic.append(semantic_id_tmp)
    probs_semantic.append(probs_tmp)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 497/497 [05:01<00:00,  1.65it/s]


In [17]:
# write_list(true_scores_semantic, f'chatgpt_true_scores_{task}_semantic.p')
write_list(occurances_semantic, f'chatgpt_occurances_{task}_semantic_fewshot.p')
write_list(semantic_ids_semantic, f'chatgpt_semantic_ids_{task}_semantic_fewshot.p')
write_list(probs_semantic, f'chatgpt_probs_{task}_semantic_fewshot.p')

Done writing list into a binary file
Done writing list into a binary file
Done writing list into a binary file


In [19]:
chatgpt_occurances = read_list(f'chatgpt_occurances_{task}_semantic_fewshot.p')
chatgpt_semantic_ids = read_list(f'chatgpt_semantic_ids_{task}_semantic_fewshot.p')
chatgpt_probs = read_list(f'chatgpt_probs_{task}_semantic_fewshot.p')

In [20]:
indices = np.arange(len(retrieved_true_scores))
random.shuffle(indices)
cal_first_indices = indices[:int(len(indices) * 0.3)]
cal_second_indices = indices[int(len(indices) * 0.3) : int(len(indices) * 0.6)]
test_indices = indices[int(len(indices) * 0.6):]

cal_first_retrieved_true_scores = utils.split(retrieved_true_scores, cal_first_indices)
cal_second_retrieved_true_scores = utils.split(retrieved_true_scores, cal_second_indices)
test_retrieved_true_scores = utils.split(retrieved_true_scores, test_indices)

cal_first_chatgpt_true_scores = utils.split(chatgpt_true_scores, cal_first_indices)
cal_second_chatgpt_true_scores = utils.split(chatgpt_true_scores, cal_second_indices)
test_chatgpt_true_scores = utils.split(chatgpt_true_scores, test_indices)

cal_first_retrieved_scores = utils.split(retrieved_scores, cal_first_indices)
cal_second_retrieved_scores = utils.split(retrieved_scores, cal_second_indices)
test_retrieved_scores = utils.split(retrieved_scores, test_indices)

cal_first_chatgpt_occurances = utils.split(chatgpt_occurances, cal_first_indices)
cal_second_chatgpt_occurances = utils.split(chatgpt_occurances, cal_second_indices)
test_chatgpt_occurances = utils.split(chatgpt_occurances, test_indices)

cal_first_chatgpt_semantic_ids = utils.split(chatgpt_semantic_ids, cal_first_indices)
cal_second_chatgpt_semantic_ids = utils.split(chatgpt_semantic_ids, cal_second_indices)
test_chatgpt_semantic_ids = utils.split(chatgpt_semantic_ids, test_indices)

cal_first_chatgpt_probs = utils.split(chatgpt_probs, cal_first_indices)
cal_second_chatgpt_probs = utils.split(chatgpt_probs, cal_second_indices)
test_chatgpt_probs = utils.split(chatgpt_probs, test_indices)

cal_first_queries = utils.split(queries, cal_first_indices)
cal_second_queries = utils.split(queries, cal_second_indices)
test_queries = utils.split(queries, test_indices)

cal_first_chatgpt_answers = utils.split(chatgpt_answers, cal_first_indices)
cal_second_chatgpt_answers = utils.split(chatgpt_answers, cal_second_indices)
test_chatgpt_answers = utils.split(chatgpt_answers, test_indices)

cal_first_answers = utils.split(answers, cal_first_indices)
cal_second_answers = utils.split(answers, cal_second_indices)
test_answers = utils.split(answers, test_indices)

In [21]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"],
                                        use_stemmer=True)

## Compute coverage rate

In [22]:
def coverage(
        retrieved_true_scores_list, opensource_true_scores_list,
        retrieved_thr, qa_thr):

    includes = []
    for idx, (retrieved_true_score, opensource_true_score) in enumerate(zip(retrieved_true_scores_list, opensource_true_scores_list)):
#         if idx > 20:
        opensource_true_score = np.max(opensource_true_score)
        include = True if (retrieved_true_score >= retrieved_thr and 
                           opensource_true_score >= qa_thr) \
                       else False
        includes.append(include)
    return includes

In [23]:
alpha = 0.3

In [24]:
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha/2)
cal_first_scores = []
for scores in cal_first_chatgpt_true_scores:
    cal_first_scores.append(np.max(scores))
chatgpt_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha/2)

retrieved_coverage = np.mean(np.array(cal_second_retrieved_true_scores) >= retrieved_thr)
cal_second_scores = []
for scores in cal_second_chatgpt_true_scores:
    cal_second_scores.append(np.max(scores))
qa_coverage = np.mean(np.array(cal_second_scores) >= chatgpt_qa_thr)
print('retrieval coverage', retrieved_coverage)
print('qa coverage', qa_coverage)

retrieved_coverage = np.mean(np.array(test_retrieved_true_scores) >= retrieved_thr)
test_scores = []
for scores in test_chatgpt_true_scores:
    test_scores.append(np.max(scores))
qa_coverage = np.mean(np.array(test_scores) >= chatgpt_qa_thr)
print('test retrieval coverage', retrieved_coverage)
print('test qa coverage', qa_coverage)

retrieval coverage 0.8590604026845637
qa coverage 0.8322147651006712
test retrieval coverage 0.8592964824120602
test qa coverage 0.8592964824120602


In [25]:
coverages = coverage(test_retrieved_true_scores, 
                    test_chatgpt_true_scores,
                    retrieved_thr,
                    chatgpt_qa_thr
                   )
print('End-to-end coverage', np.mean(coverages))

End-to-end coverage 0.7336683417085427


## PAC-Bonf

In [26]:
from pac_utils import find_maximum_train_error_allow

In [27]:
delta = 0.1
retrieve_alpha = find_maximum_train_error_allow(alpha/2.0, delta/2.0, len(cal_first_indices))
qa_alpha = find_maximum_train_error_allow(alpha/2.0, delta/2.0, len(cal_first_indices))
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=retrieve_alpha)
cal_first_scores = []
for scores in cal_first_chatgpt_true_scores:
    cal_first_scores.append(np.max(scores))
chatgpt_qa_thr = utils.compute_threshold(cal_first_scores, alpha=qa_alpha)

retrieved_coverage = np.mean(np.array(cal_second_retrieved_true_scores) >= retrieved_thr)
cal_second_scores = []
for scores in cal_second_chatgpt_true_scores:
    cal_second_scores.append(np.max(scores))
qa_coverage = np.mean(np.array(cal_second_scores) >= chatgpt_qa_thr)
print('retrieval coverage', retrieved_coverage)
print('qa coverage', qa_coverage)

retrieved_coverage = np.mean(np.array(test_retrieved_true_scores) >= retrieved_thr)
test_scores = []
for scores in test_chatgpt_true_scores:
    test_scores.append(np.max(scores))
qa_coverage = np.mean(np.array(test_scores) >= chatgpt_qa_thr)
print('test retrieval coverage', retrieved_coverage)
print('test qa coverage', qa_coverage)

retrieval coverage 0.9328859060402684
qa coverage 0.9530201342281879
test retrieval coverage 0.8894472361809045
test qa coverage 0.9396984924623115


In [28]:
coverages = coverage(test_retrieved_true_scores, 
                    test_chatgpt_true_scores,
                    retrieved_thr,
                    chatgpt_qa_thr
                   )
print('End-to-end coverage', np.mean(coverages))

End-to-end coverage 0.8291457286432161


## Bayesian optimization

In [33]:
from skopt.space import Real
from skopt import gp_minimize
from skopt.utils import use_named_args
import multiprocessing
from multiprocessing import Value

In [35]:
def evaluate(
        test_retrieved_scores,
        test_queries, test_answers, test_chatgpt_answers, 
        test_chatgpt_semantic_ids, test_chatgpt_probs,
        retrieved_thr, chatgpt_qa_thr,
        cluster=True, kernel=40, verbose=True):
    
    length = len(test_retrieved_scores)
    lens = np.linspace(0, length, kernel+1)
    test_retrieved_scores_list = [test_retrieved_scores[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_chatgpt_semantic_ids_list = [test_chatgpt_semantic_ids[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_chatgpt_probs_list = [test_chatgpt_probs[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_answers_list = [test_answers[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    
    def run(i, shared_includes, shared_semantic_counts):
        includes = []
        semantics_total = []
        coverages = []
        semantic_counts = []
        for idx, (retrieved_scores_tmp, answers_tmp,\
                chatgpt_semantic_ids_tmp, chatgpt_probs_tmp) \
            in tqdm(enumerate(zip(
                test_retrieved_scores_list[i], test_answers_list[i], \
                test_chatgpt_semantic_ids_list[i], test_chatgpt_probs_list[i])), total=len(test_retrieved_scores_list[i])):

            include = False
    #         coverage_tmp = coverage(retrieved_true_scores_tmp, 
    #                         [chatgpt_true_scores_tmp],
    #                         retrieved_thr,
    #                         chatgpt_qa_thr
    #                        )[0]
    #         coverages.append(coverage_tmp)
            retrieved_count = 0
            semantics = []
            for retrieved_score, semantic_set_ids, probs in zip(retrieved_scores_tmp, chatgpt_semantic_ids_tmp, chatgpt_probs_tmp):
                    if retrieved_score < retrieved_thr:
                        continue
                    else:
                        retrieved_count += 1
                        for predicted_answer in semantic_set_ids.keys():
                            concept_id = semantic_set_ids[predicted_answer]
                            prob = probs[concept_id]
                            if prob >= chatgpt_qa_thr:
                                semantics.append(predicted_answer)

                                # TODO: check if the semantic is consistent with true answer
                                if include is False:
                                    for answer_tmp in answers_tmp:
                                        scores = scorer.score(answer_tmp,
                                                              predicted_answer)
                                        scores = scores['rouge1'][2]
                                        if scores >= 0.3:
                                            include = True
                                            break
            if cluster:
                semantic_set_ids, semantic_probs, item_occurance = \
                            utils.clustering(semantics, "", scorer=scorer)
                semantic_counts.append(len(semantic_probs.keys()))
            else:
                semantic_counts.append(len(semantics))
            semantics_total.append(semantic_set_ids)
    #         answer_counts.append(retrieved_count)
            includes.append(include)
        shared_includes.value += np.sum(includes)
        shared_semantic_counts.value += np.sum(semantic_counts)
    
    processes = []
    shared_includes = Value('f', 0.0)
    shared_semantic_counts = Value('f', 0.0)

    for i in range(0, kernel):
        p = multiprocessing.Process(target=run, args=(i, shared_includes, shared_semantic_counts))
        processes.append(p)
        p.start()

    for process in processes:
        process.join()
        
    return shared_includes.value/length, 0.0, shared_semantic_counts.value/length

## Bonf method

In [31]:
def softmax(vec):
    nom = np.exp(vec - np.mean(vec))
    return nom / np.sum(nom)

In [ ]:
"""
Weight Bonf module
"""
w1 = Real(name='w1', low=0.0, high=1.0)
w2 = Real(name='w2', low=0.0, high=1.0)

# Gather the search-space dimensions in a list.
dimensions = [w1, w2]

@use_named_args(dimensions=dimensions)
def objective(w1, w2):
    weights = softmax(np.array([w1, w2])).reshape(-1, 1)
    alpha_retrieve = alpha * weights[0]
    alpha_qa = alpha * weights[1]
    retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve)
    cal_first_scores = []
    for scores in cal_first_chatgpt_true_scores:
        cal_first_scores.append(np.max(scores))
    chatgpt_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa)
    results = evaluate(cal_second_retrieved_scores, cal_second_queries, 
                       cal_second_answers, cal_second_chatgpt_answers, \
                       cal_second_chatgpt_semantic_ids,cal_second_chatgpt_probs, 
                       retrieved_thr, chatgpt_qa_thr, 
                       cluster=False)
    coverage = np.mean(results[0])
    average_answer = np.mean(results[1])
    average_semantic = np.mean(results[2])
    return average_semantic

result = gp_minimize(func=objective,
                     dimensions=dimensions,
                     acq_func="EI",      # the acquisition function
                     n_calls=15,
                     random_state=seed,
                     verbose=True,
                     x0=[[1, 1]])

print("Best fitness:", result.fun)
print("Best parameters:", softmax(result.x))

Iteration No: 1 started. Evaluating function at provided point.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 30.03it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 54.99it/s]


Iteration No: 1 ended. Evaluation done at provided point.
Time taken: 3.5203
Function value obtained: 119.9864
Current minimum: 119.9864
Iteration No: 2 started. Evaluating function at random point.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 71.82it/s]


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 3.2420
Function value obtained: 118.7288
Current minimum: 118.7288
Iteration No: 3 started. Evaluating function at random point.


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 41.04it/s]


Iteration No: 10 ended. Evaluation done at random point.
Time taken: 3.7059
Function value obtained: 121.9898
Current minimum: 118.7288
Iteration No: 11 started. Evaluating function at random point.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 55.39it/s]


Iteration No: 11 ended. Evaluation done at random point.
Time taken: 3.8651
Function value obtained: 118.1593
Current minimum: 118.1593
Iteration No: 12 started. Evaluating function at random point.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 24.29it/s]


In [36]:
%%time
weights = softmax(result.x).reshape(-1, 1)
alpha_retrieve = alpha * weights[0]
alpha_qa = alpha * weights[1]
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve)
cal_first_scores = []
for scores in cal_first_chatgpt_true_scores:
    cal_first_scores.append(np.max(scores))
chatgpt_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa)
results = evaluate(
    test_retrieved_scores, test_queries,
    test_answers, test_chatgpt_answers, 
    test_chatgpt_semantic_ids, test_chatgpt_probs, 
    retrieved_thr, chatgpt_qa_thr,
    cluster=False)

NameError: name 'result' is not defined

In [ ]:
print('Desired coverage rate', 1-alpha)
print('Coverage', np.mean(results[0]))
# print('Average answer', np.mean(results[1]))
print('Average semantic', np.mean(results[2]))

In [37]:
%%time
alpha_retrieve = alpha * (1/2.0)
alpha_qa = alpha * (1/2.0)
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve)
cal_first_scores = []
for scores in cal_first_chatgpt_true_scores:
    cal_first_scores.append(np.max(scores))
chatgpt_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa)
results = evaluate(
    test_retrieved_scores, test_queries,
    test_answers, test_chatgpt_answers, 
    test_chatgpt_semantic_ids, test_chatgpt_probs,
    retrieved_thr, chatgpt_qa_thr,
    cluster=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.38it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.81s/it]


CPU times: user 105 ms, sys: 1.34 s, total: 1.45 s
Wall time: 20.2 s


In [38]:
print('Desired coverage rate', 1-alpha)
print('Coverage', np.mean(results[0]))
# print('Average answer', np.mean(results[1]))
print('Average semantic', np.mean(results[2]))

Desired coverage rate 0.7
Coverage 0.8391959798994975
Average semantic 6.507537688442211


In [39]:
def evaluate_vanila(
        test_retrieved_scores,
        test_queries, test_answers, test_chatgpt_answers, 
        test_chatgpt_semantic_ids, test_chatgpt_probs,
        cluster=True, kernel=40, verbose=True):
    
    length = len(test_retrieved_scores)
    lens = np.linspace(0, length, kernel+1)
    test_retrieved_scores_list = [test_retrieved_scores[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_chatgpt_semantic_ids_list = [test_chatgpt_semantic_ids[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_chatgpt_probs_list = [test_chatgpt_probs[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    test_answers_list = [test_answers[int(lens[i]):int(lens[i+1])] for i in range(kernel)]
    
    def run(i, shared_includes, shared_semantic_counts):
        includes = []
        semantics_total = []
        coverages = []
        semantic_counts = []
        for idx, (retrieved_scores_tmp, answers_tmp,\
                chatgpt_semantic_ids_tmp, chatgpt_probs_tmp) \
            in tqdm(enumerate(zip(
                test_retrieved_scores_list[i], test_answers_list[i], \
                test_chatgpt_semantic_ids_list[i], test_chatgpt_probs_list[i])), total=len(test_retrieved_scores_list[i])):

            include = False
    #         coverage_tmp = coverage(retrieved_true_scores_tmp, 
    #                         [chatgpt_true_scores_tmp],
    #                         retrieved_thr,
    #                         chatgpt_qa_thr
    #                        )[0]
    #         coverages.append(coverage_tmp)
            retrieved_count = 0
            semantics = []
            for retrieved_score, semantic_set_ids, probs in zip(retrieved_scores_tmp, chatgpt_semantic_ids_tmp, chatgpt_probs_tmp):
                retrieved_count += 1
                for predicted_answer in semantic_set_ids.keys():
                    concept_id = semantic_set_ids[predicted_answer]
                    prob = probs[concept_id]
                    semantics.append(predicted_answer)
                    if include is False:
                        for answer_tmp in answers_tmp:
                            scores = scorer.score(answer_tmp,
                                                  predicted_answer)
                            scores = scores['rouge1'][2]
                            if scores >= 0.3:
                                include = True
                                break
                break
            if cluster:
                semantic_set_ids, semantic_probs, item_occurance = \
                            utils.clustering(semantics, "", scorer=scorer)
                semantic_counts.append(len(semantic_probs.keys()))
            else:
                semantic_counts.append(len(semantics))
            semantics_total.append(semantic_set_ids)
    #         answer_counts.append(retrieved_count)
            includes.append(include)
        shared_includes.value += np.sum(includes)
        shared_semantic_counts.value += np.sum(semantic_counts)
    
    processes = []
    shared_includes = Value('f', 0.0)
    shared_semantic_counts = Value('f', 0.0)

    for i in range(0, kernel):
        p = multiprocessing.Process(target=run, args=(i, shared_includes, shared_semantic_counts))
        processes.append(p)
        p.start()

    for process in processes:
        process.join()
        
    return shared_includes.value/length, 0.0, shared_semantic_counts.value/length

In [40]:
%%time
results = evaluate_vanila(
    test_retrieved_scores, test_queries,
    test_answers, test_chatgpt_answers, 
    test_chatgpt_semantic_ids, test_chatgpt_probs, 
    cluster=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 25.60it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 44.48it/s]


CPU times: user 67.1 ms, sys: 1.56 s, total: 1.63 s
Wall time: 1.35 s


In [41]:
print('Desired coverage rate', 1-alpha)
print('Coverage', np.mean(results[0]))
# print('Average answer', np.mean(results[1]))
print('Average semantic', np.mean(results[2]))

Desired coverage rate 0.7
Coverage 0.7185929648241206
Average semantic 6.110552763819095


## PAC-Bonf method

In [42]:
alpha_retrieve = alpha * (1/2.0)
alpha_qa = alpha * (1/2.0)

delta = 0.1
alpha_retrieve_pac = find_maximum_train_error_allow(alpha_retrieve, delta/2.0, len(cal_first_indices))
alpha_qa_pac = find_maximum_train_error_allow(alpha_qa, delta/2.0, len(cal_first_indices))

retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve_pac)
cal_first_scores = []
for scores in cal_first_chatgpt_true_scores:
    cal_first_scores.append(np.max(scores))
chatgpt_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa_pac)
results = evaluate(
    test_retrieved_scores, test_queries,
    test_answers, test_chatgpt_answers, 
    test_chatgpt_semantic_ids, test_chatgpt_probs,
    retrieved_thr, chatgpt_qa_thr,
    cluster=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.24s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:52<00:00, 10.48s/it]


In [43]:
print('Coverage', np.mean(results[0]))
# print('Average answer', np.mean(results[1]))
print('Average semantic', np.mean(results[2]))

Coverage 0.9095477386934674
Average semantic 12.346733668341708
